<a href="https://colab.research.google.com/github/Phung-Thu-Trang/Group8-final-project/blob/trang/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Cài đặt PySpark nếu chưa có
!pip install pyspark


In [20]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col, mean, stddev, when, count


In [21]:
# Khởi tạo SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IBM Stock Data") \
    .getOrCreate()


In [22]:
from google.colab import files
uploaded = files.upload()


Saving IBM.csv to IBM (2).csv


In [23]:
df = spark.read.csv("IBM.csv", header=True, inferSchema=True)
# In ra cấu trúc (schema) của dataframe
df.printSchema()
df.describe().show()# Thống kê mô tả (mean, stddev, min, max)
df.show(5)


root
 |-- date: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)

+-------+---------+------------------+------------------+------------------+-----------------+------------------+
|summary|     date|              open|              high|               low|            close|            volume|
+-------+---------+------------------+------------------+------------------+-----------------+------------------+
|  count|    15946|             15946|             15946|             15946|            15946|             15946|
|   mean|     NULL| 40.43062425059552|40.802488680546766|40.077803580835365|40.44940021949066|1825068.3348175092|
| stddev|     NULL|47.238127846900696|  47.6467088937108|46.850355960161465|47.27480817329008|  2169698.86385416|
|    min|1/10/1962|            0.8127|            0.8538|            0.7967|           0.81

In [24]:
# Kiểm tra null
#from pyspark.sql.functions import col

df.select([col(c).isNull().alias(c) for c in df.columns]).show()

# Đếm số giá trị null trong từng cột
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()


# Loại bỏ các dòng bị null (nếu có)
df_clean = df.dropna()

# Chuyển đổi cột 'Date' sang định dạng ngày
df_clean = df_clean.withColumn("date", to_date(col("date"), "M/d/yyyy"))

# Điền giá trị trung bình cho các cột số bị thiếu
numeric_cols = ["close", "volume", "open", "high", "low"]
df = df.fillna({col_name: df.select(mean(col(col_name))).collect()[0][0] for col_name in numeric_cols})

# Sắp xếp ngày theo thứ tự
df_pd = df_clean.select("Date", "Close").orderBy("Date").toPandas()





df_clean.show(5)

+-----+-----+-----+-----+-----+------+
| date| open| high|  low|close|volume|
+-----+-----+-----+-----+-----+------+
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
|false|false|false|false|false| false|
+-----+-----+-----+-----+-----+------+
only showing top 20 rows

+----+----+----+---+-----+------+
|dat

In [25]:
# Chuyển DataFrame PySpark thành pandas và ghi ra file
df_clean.toPandas().to_csv("IBM_clean.csv", index=False)

# Cho phép tải về file từ Colab
from google.colab import files
files.download("IBM_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>